# Jacobi 恒等式教程

## 学习目标

本 notebook 将带你深入理解顶点算符代数 (Vertex Operator Algebra, VOA) 中的 **Jacobi 恒等式**，这是 VOA 理论的核心恒等式之一。

**学习目标**：
1. 理解 Jacobi 恒等式的数学定义和物理意义
2. 掌握使用 pyope 验证 Jacobi 恒等式的方法
3. 通过具体例子（Virasoro 代数、流代数）加深理解
4. 学会解读 Jacobi 恒等式的验证结果

**前置知识**：
- 基本的顶点算符代数概念
- 算符积展开 (OPE) 的定义
- Python 和 SymPy 基础

**参考资料**：
- V. Kac, "Vertex Algebras for Beginners", Chapter 3
- `tests/JACOBI_README.md`
- `src/pyope/jacobi.py`

## 1. 数学背景：Jacobi 恒等式

### 1.1 定义

在顶点算符代数中，Jacobi 恒等式描述了三个算符之间的结合律关系：

$$
\{A\{ BC\}_q\}_p - (-1)^{|A||B|} \{B \{AC\}_p\}_q - \sum_{\ell \ge 1} \binom{p-1}{\ell-1} \{\{AB\}_{\ell}C\}_{p + q - \ell} = 0
$$

其中：
- $\{AB\}_n$ 表示算符 $A$ 和 $B$ 的 OPE 中 $(z-w)^{-n}$ 项的系数（称为 bracket）
- $|A|, |B|$ 表示算符的 parity（0 表示玻色子，1 表示费米子）
- $\binom{p-1}{\ell-1}$ 是二项式系数

### 1.2 物理意义

Jacobi 恒等式保证了：
1. **结合律**：OPE 计算的结果与计算顺序无关
2. **代数一致性**：确保顶点算符代数的结构是自洽的
3. **对称性**：体现了算符之间的对称关系

如果一个 OPE 系统满足 Jacobi 恒等式，那么它就构成了一个合法的顶点算符代数。

## 2. 环境设置

首先导入必要的模块：

In [ ]:
# 导入 pyope 核心模块
from pyope import (
    BasisOperator,           # 基本算符
    OPE,                     # OPE 计算器
    d, dn,                   # 导数函数
    One, Zero,               # 常数算符
    Bosonic, Fermionic,      # 算符类型声明
    check_jacobi_identity,   # 检查 Jacobi 恒等式（返回矩阵）
    verify_jacobi_identity,  # 验证 Jacobi 恒等式（返回布尔值）
    bracket,                 # bracket 函数
)

# 导入 sympy 用于符号计算
import sympy as sp
from sympy import symbols, simplify, expand

# 启用 LaTeX 渲染
sp.init_printing(use_latex='mathjax')

print("✓ 模块导入成功！")

## 3. 基础示例：Virasoro 代数

### 3.1 定义 Virasoro 代数

Virasoro 代数是共形场论中最重要的代数结构之一。能动张量 $T(z)$ 的 OPE 为：

$$
T(z)T(w) = \frac{c/2}{(z-w)^4} + \frac{2T(w)}{(z-w)^2} + \frac{\partial T(w)}{z-w} + \text{regular}
$$

其中 $c$ 是中心荷 (central charge)。

In [ ]:
# 定义符号
c = sp.Symbol('c')  # 中心荷

# 定义 Virasoro 算符 T，共形权重为 2
T = BasisOperator("T", bosonic=True, conformal_weight=2)
Bosonic(T)  # 声明 T 是玻色算符

print(f"✓ 定义了能动张量 T")
print(f"  - 共形权重: {T.conformal_weight}")
print(f"  - 类型: 玻色算符")

In [ ]:
# 定义 T 的 OPE
# T(z)T(w) = c/2/(z-w)^4 + 2T(w)/(z-w)^2 + ∂T(w)/(z-w)
OPE[T, T] = OPE.make([
    c/2 * One,    # 4阶极点：c/2
    0,            # 3阶极点：无
    2*T,          # 2阶极点：2T
    d(T)          # 1阶极点：∂T
])

print("✓ 定义了 Virasoro OPE:")
print("  T(z)T(w) = (c/2)/(z-w)^4 + 2T/(z-w)^2 + ∂T/(z-w)")

### 3.2 验证 OPE 结构

让我们先验证 OPE 的定义是否正确：

In [ ]:
# 计算 OPE(T, T)
ope_TT = OPE(T, T)

print("OPE(T, T) 的结构:")
print(f"  最高极点: {ope_TT.max_pole}")
print(f"\n各阶极点的系数:")
for n in range(ope_TT.max_pole, 0, -1):
    coeff = ope_TT.pole(n)
    if coeff != 0:
        print(f"  pole({n}): {coeff}")

# 显示 LaTeX 渲染
print("\nLaTeX 渲染:")
display(ope_TT.pole(4))
display(ope_TT.pole(2))
display(ope_TT.pole(1))

### 3.3 使用 `check_jacobi_identity` 检查 Jacobi 恒等式

`check_jacobi_identity(A, B, C)` 函数计算 Jacobi 恒等式的左侧，返回一个矩阵。

矩阵的第 $(m, n)$ 个元素对应于 Jacobi 恒等式在特定 $(m, n)$ 处的值。

如果 Jacobi 恒等式成立，所有矩阵元素应该为 0。

In [ ]:
# 检查 Jacobi 恒等式 check_jacobi_identity(T, T, T)
print("计算 Jacobi 恒等式: check_jacobi_identity(T, T, T)...")
print("这可能需要几秒钟...\n")

jacobi_result = check_jacobi_identity(T, T, T, simplify_func=sp.expand)

print(f"✓ 计算完成！")
print(f"结果矩阵维度: {len(jacobi_result)} × {len(jacobi_result[0])}")

In [ ]:
# 检查结果
print("检查 Jacobi 恒等式结果:\n")

all_zero = True
for i, row in enumerate(jacobi_result):
    for j, value in enumerate(row):
        if value != 0:
            print(f"  非零元素 [{i+1},{j+1}]: {value}")
            all_zero = False

if all_zero:
    print("  ✓ 所有元素都为 0！")
    print("  ✓ Jacobi 恒等式成立！")
else:
    print("  ✗ 存在非零元素")
    print("  ✗ Jacobi 恒等式不成立")

### 3.4 使用 `verify_jacobi_identity` 便捷验证

`verify_jacobi_identity(A, B, C)` 是一个便捷函数，直接返回布尔值：

In [ ]:
# 使用便捷函数验证
result = verify_jacobi_identity(T, T, T, simplify_func=sp.expand)

print(f"verify_jacobi_identity(T, T, T) = {result}")

if result:
    print("\n✓ Virasoro 代数满足 Jacobi 恒等式！")
else:
    print("\n✗ Virasoro 代数不满足 Jacobi 恒等式")

### 3.5 理解结果矩阵

让我们可视化结果矩阵：

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 将结果转换为数值矩阵（用于可视化）
# 由于所有元素都是 0，我们只是展示矩阵结构
matrix_shape = (len(jacobi_result), len(jacobi_result[0]))
matrix_data = np.zeros(matrix_shape)

# 创建热图
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(matrix_data, cmap='RdYlGn_r', aspect='auto')

# 设置标签
ax.set_xlabel('n (列索引)', fontsize=12)
ax.set_ylabel('m (行索引)', fontsize=12)
ax.set_title('Jacobi 恒等式结果矩阵 (T, T, T)', fontsize=14, fontweight='bold')

# 添加网格
ax.set_xticks(np.arange(matrix_shape[1]))
ax.set_yticks(np.arange(matrix_shape[0]))
ax.set_xticklabels(np.arange(1, matrix_shape[1]+1))
ax.set_yticklabels(np.arange(1, matrix_shape[0]+1))
ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.5)

# 添加颜色条
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('值', rotation=270, labelpad=15)

plt.tight_layout()
plt.show()

print(f"\n说明: 矩阵的所有元素都是 0（绿色），表示 Jacobi 恒等式成立。")

## 4. 进阶示例：简单流代数 (Current Algebra)

### 4.1 定义流算符

考虑一个简单的 U(1) 流代数，流算符 $J(z)$ 的 OPE 为：

$$
J(z)J(w) = \frac{k}{(z-w)^2} + \text{regular}
$$

其中 $k$ 是水平 (level)。

In [ ]:
# 定义流算符 J，共形权重为 1
J = BasisOperator("J", bosonic=True, conformal_weight=1)
Bosonic(J)

# 定义水平
k = sp.Symbol('k')

# 定义 J 的 OPE: J(z)J(w) = k/(z-w)^2
OPE[J, J] = OPE.make([
    k * One,      # 2阶极点：k
    0             # 1阶极点：无
])

print("✓ 定义了流算符 J")
print(f"  - 共形权重: {J.conformal_weight}")
print(f"  - OPE: J(z)J(w) = k/(z-w)^2")

### 4.2 验证流代数的 Jacobi 恒等式

In [ ]:
# 验证 Jacobi 恒等式
print("验证流代数的 Jacobi 恒等式: verify_jacobi_identity(J, J, J)...\n")

result_J = verify_jacobi_identity(J, J, J, simplify_func=sp.expand)

print(f"结果: {result_J}")

if result_J:
    print("\n✓ 流代数满足 Jacobi 恒等式！")
else:
    print("\n✗ 流代数不满足 Jacobi 恒等式")

In [ ]:
# 查看详细的结果矩阵
jacobi_J = check_jacobi_identity(J, J, J, simplify_func=sp.expand)

print(f"结果矩阵维度: {len(jacobi_J)} × {len(jacobi_J[0])}")
print("\n矩阵内容:")
for i, row in enumerate(jacobi_J):
    print(f"  行 {i+1}: {row}")

## 5. 交互式探索：修改参数

### 5.1 改变中心荷

让我们看看改变 Virasoro 代数的中心荷 $c$ 是否会影响 Jacobi 恒等式：

In [ ]:
# 测试不同的中心荷值
test_values = [1, 2, sp.Rational(1, 2), sp.Symbol('c_test')]

print("测试不同的中心荷值:\n")

for c_val in test_values:
    # 创建新的算符
    T_test = BasisOperator(f"T_{c_val}", bosonic=True, conformal_weight=2)
    Bosonic(T_test)
    
    # 定义 OPE
    OPE[T_test, T_test] = OPE.make([
        c_val/2 * One,
        0,
        2*T_test,
        d(T_test)
    ])
    
    # 验证 Jacobi 恒等式
    result = verify_jacobi_identity(T_test, T_test, T_test, simplify_func=sp.expand)
    
    status = "✓" if result else "✗"
    print(f"  {status} c = {c_val}: Jacobi 恒等式 {'成立' if result else '不成立'}")

print("\n结论: Jacobi 恒等式对任意中心荷都成立！")

### 5.2 错误的 OPE 定义

让我们看看如果 OPE 定义错误，Jacobi 恒等式会失败：

In [ ]:
# 定义一个错误的 OPE（缺少 ∂T 项）
T_wrong = BasisOperator("T_wrong", bosonic=True, conformal_weight=2)
Bosonic(T_wrong)

# 错误的 OPE: 缺少 1阶极点
OPE[T_wrong, T_wrong] = OPE.make([
    c/2 * One,    # 4阶极点
    0,            # 3阶极点
    2*T_wrong,    # 2阶极点
    0             # 1阶极点：错误！应该是 d(T_wrong)
])

print("定义了错误的 OPE（缺少 ∂T 项）")
print("验证 Jacobi 恒等式...\n")

result_wrong = verify_jacobi_identity(T_wrong, T_wrong, T_wrong, simplify_func=sp.expand)

print(f"结果: {result_wrong}")

if not result_wrong:
    print("\n✓ 如预期，错误的 OPE 不满足 Jacobi 恒等式！")
    print("\n让我们看看哪些项不为零:")
    
    jacobi_wrong = check_jacobi_identity(T_wrong, T_wrong, T_wrong, simplify_func=sp.expand)
    
    for i, row in enumerate(jacobi_wrong):
        for j, value in enumerate(row):
            if value != 0:
                print(f"  非零元素 [{i+1},{j+1}]: {value}")

## 6. API 使用指南

### 6.1 `check_jacobi_identity` vs `verify_jacobi_identity`

pyope 提供了两个函数来检查 Jacobi 恒等式：

| 函数 | 返回值 | 用途 |
|------|--------|------|
| `check_jacobi_identity(A, B, C)` | 二维列表（矩阵） | 获取详细的计算结果，用于调试 |
| `verify_jacobi_identity(A, B, C)` | 布尔值 | 快速验证是否成立 |

### 6.2 使用 `simplify_func` 参数

`simplify_func` 参数用于简化计算结果。常用选项：

In [ ]:
# 示例：不同的简化函数
from sympy import simplify, expand, factor

print("测试不同的简化函数:\n")

# 1. 使用 expand（默认）
result1 = verify_jacobi_identity(T, T, T, simplify_func=expand)
print(f"  expand: {result1}")

# 2. 使用 simplify
result2 = verify_jacobi_identity(T, T, T, simplify_func=simplify)
print(f"  simplify: {result2}")

# 3. 不使用简化
result3 = verify_jacobi_identity(T, T, T, simplify_func=None)
print(f"  None: {result3}")

print("\n说明: 对于 Virasoro 代数，所有简化方法都给出相同结果。")

### 6.3 解读结果矩阵

结果矩阵的维度和含义：

In [ ]:
# 获取结果矩阵
jacobi_matrix = check_jacobi_identity(T, T, T, simplify_func=expand)

print("结果矩阵解读:\n")
print(f"  维度: {len(jacobi_matrix)} × {len(jacobi_matrix[0])}")
print(f"  行索引 m: 1 到 {len(jacobi_matrix)}")
print(f"  列索引 n: 1 到 {len(jacobi_matrix[0])}")
print(f"\n  矩阵元素 [m, n] 对应 Jacobi 恒等式在 (m, n) 处的值")
print(f"  如果所有元素为 0，则 Jacobi 恒等式成立")

print(f"\n示例: 访问特定元素")
print(f"  jacobi_matrix[0][0] (m=1, n=1): {jacobi_matrix[0][0]}")
print(f"  jacobi_matrix[1][1] (m=2, n=2): {jacobi_matrix[1][1]}")

## 7. 理论背景（可选）

### 7.1 Jacobi 恒等式的推导

Jacobi 恒等式可以从 OPE 的结合律推导出来。考虑三个算符 $A(z)$, $B(w)$, $C(u)$ 的乘积：

$$
A(z)B(w)C(u)
$$

我们可以按不同顺序计算 OPE：
1. 先计算 $B(w)C(u)$，再与 $A(z)$ 做 OPE
2. 先计算 $A(z)C(u)$，再与 $B(w)$ 做 OPE
3. 先计算 $A(z)B(w)$，再与 $C(u)$ 做 OPE

Jacobi 恒等式保证了这三种计算方式给出相同的结果。

### 7.2 与 OPE 结合律的关系

在算符形式语言中，Jacobi 恒等式等价于：

$$
[A_m, [B_n, C]] = [B_n, [A_m, C]] + [[A_m, B_n], C]
$$

这是李代数的 Jacobi 恒等式的推广。

### 7.3 物理意义

在共形场论中，Jacobi 恒等式确保了：
- **因果性**：算符在不同时空点的乘积是良定义的
- **对称性**：代数结构与物理对称性一致
- **Ward 恒等式**：对称性生成元满足正确的代数关系

## 8. 练习与探索

### 练习 1：验证混合算符的 Jacobi 恒等式

定义 $T$ 和 $J$ 的混合 OPE，验证 `check_jacobi_identity(T, J, J)`：

In [ ]:
# 定义 T 和 J 的混合 OPE
# T(z)J(w) = J(w)/(z-w)^2 + ∂J(w)/(z-w)
OPE[T, J] = OPE.make([
    J,          # 2阶极点
    d(J)        # 1阶极点
])

print("✓ 定义了混合 OPE: T(z)J(w) = J/(z-w)^2 + ∂J/(z-w)")
print("\n验证 Jacobi 恒等式 check_jacobi_identity(T, J, J)...\n")

result_TJJ = verify_jacobi_identity(T, J, J, simplify_func=expand)

print(f"结果: {result_TJJ}")

if result_TJJ:
    print("\n✓ 混合算符满足 Jacobi 恒等式！")
else:
    print("\n✗ 混合算符不满足 Jacobi 恒等式")

### 练习 2：自定义算符

尝试定义你自己的算符和 OPE，验证 Jacobi 恒等式：

In [ ]:
# 你的代码：
# 1. 定义一个新的算符
# 2. 定义它的 OPE
# 3. 验证 Jacobi 恒等式

# 示例：定义一个共形权重为 3 的算符 W
W = BasisOperator("W", bosonic=True, conformal_weight=3)
Bosonic(W)

# 定义简单的 OPE: W(z)W(w) = 1/(z-w)^6
OPE[W, W] = OPE.make([
    One,  # 6阶极点
    0,    # 5阶极点
    0,    # 4阶极点
    0,    # 3阶极点
    0,    # 2阶极点
    0     # 1阶极点
])

# 验证
result_W = verify_jacobi_identity(W, W, W, simplify_func=expand)
print(f"verify_jacobi_identity(W, W, W) = {result_W}")

## 9. 总结

### 9.1 关键要点

1. **Jacobi 恒等式**是顶点算符代数的核心恒等式，保证了 OPE 的结合律
2. **pyope 提供两个函数**：
   - `check_jacobi_identity(A, B, C)`: 返回详细矩阵
   - `verify_jacobi_identity(A, B, C)`: 返回布尔值
3. **Virasoro 代数和流代数**都满足 Jacobi 恒等式
4. **错误的 OPE 定义**会导致 Jacobi 恒等式失败

### 9.2 进一步学习

- 阅读 `tests/test_jacobi_virasoro.py` 了解更多测试用例
- 查看 `src/pyope/jacobi.py` 了解实现细节
- 参考 V. Kac 的书籍深入理解理论背景
- 探索 `demo/pyope_ope_demo.ipynb` 了解更多 OPE 计算功能

### 9.3 参考文献

1. V. Kac, "Vertex Algebras for Beginners", 2nd Edition, AMS, 1998
2. E. Frenkel and D. Ben-Zvi, "Vertex Algebras and Algebraic Curves", AMS, 2004
3. OPEdefs.m - Mathematica 实现（参考 `OPEdefs/` 目录）
4. pyope 文档 - `README.md` 和 `tests/JACOBI_README.md`

---

## 附录：技术细节

### A.1 实现细节

pyope 的 Jacobi 恒等式实现基于 Mathematica 的 OPEdefs.m（第 1601-1637 行）。

核心算法：
1. 计算基本 OPE: `OPE(A, B)`, `OPE(B, C)`, `OPE(A, C)`
2. 计算复合 OPE: `OPE(A, {BC}_n)`, `OPE(B, {AC}_n)`, `OPE({AB}_n, C)`
3. 对每个 $(m, n)$，计算 Jacobi 恒等式的三项
4. 简化并检查结果是否为 0

### A.2 性能考虑

- 对于简单算符（如 Virasoro, 流代数），计算通常在 1 秒内完成
- 对于复杂算符或高共形权重，可能需要更长时间
- 使用 `simplify_func=expand` 通常比 `simplify` 更快

### A.3 已知限制

- 目前仅支持有限阶极点的 OPE
- 符号简化可能不完全（依赖于 SymPy）
- 对于非常大的矩阵，内存使用可能较高

---

**感谢使用 pyope！**

如有问题或建议，请访问项目仓库或联系开发者。

**版本**: pyope 0.1.0  
**日期**: 2026-01-07  
**作者**: pyope 开发团队